# Multiprocessing
https://morvanzhou.github.io/tutorials/python-basic/multiprocessing/1-why/

In [16]:
import multiprocessing as mp
import threading as td
import time

## 1 add processs

In [4]:
# 与threading相似
def job(a,d):
    print('aaaaa')
    
if __name__ =='__main__':
    # t1 = td.Thread(target=job, args=(1,2))
    p1 = mp.Process(target=job, args=(1,2))

    # t1.start()
    p1.start()

    # t1.join()
    p1.join()

aaaaa


## 2 存储进程输出Queue

In [12]:
def job(q):
    '''
    job 不能有return，结果存储在Queue对象中，并且Queue要作为参数传入
    Args:
        q (Queue): Queue object for output
    '''
    res = 0
    for i in range(1000):
        res += i+i**2 + i**3
    q.put(res) # queue

if __name__ == '__main__':
    q = mp.Queue()
    p1 = mp.Process(target=job, args=(q,)) #只有一个参数也必须有逗号
    p2 = mp.Process(target=job, args=(q,))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(q)
    res1 = q.get()
    res2 = q.get()
    print(res1+res2)

499667166000


## 3.进程池

In [14]:
def job(x):
    return x*x

def multicore():
    pool = mp.Pool()
    res = pool.map(job, range(10))
    print(res)
    res = pool.apply_async(job, (2,))
    # 用get获得结果
    print(res.get())
    # 迭代器，i=0时apply一次，i=1时apply一次等等
    multi_res = [pool.apply_async(job, (i,)) for i in range(10)]
    # 从迭代器中取出
    print([res.get() for res in multi_res])
if __name__ == '__main__':
    multicore()

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
4
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


## 4. shared memory

In [15]:
value = mp.Value('d',1) #需要定义数值形式。例如d
array = mp.Array('i', [1,2,3])

## 5. 进程锁 lock

In [19]:
def job(v, num, l):
    l.acquire()
    for _ in range(5):
        time.sleep(0.1) # 暂停0.1秒，让输出效果更明显
        v.value += num # v.value获取共享变量值
        print(v.value)#, end="")
    l.release()
def multicore():
    l = mp.Lock()
    v = mp.Value('i', 0) # 定义共享变量
    p1 = mp.Process(target=job, args=(v,1,l))
    p2 = mp.Process(target=job, args=(v,3,l)) # 设定不同的number看如何抢夺内存
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    
if __name__ == '__main__':
    multicore()

1
2
3
4
5
8
11
14
17
20
